In [1]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sbs
import geopandas as gpd
import folium

In [2]:
from pyspark.sql import SparkSession

# Create a spark session
spark = (
    SparkSession.builder.appName("ADS Project1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)


24/08/22 18:48:02 WARN Utils: Your hostname, Hanshis-Laptop.local resolves to a loopback address: 127.0.0.1; using 10.12.218.96 instead (on interface en0)
24/08/22 18:48:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/22 18:48:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.conf.set("spark.sql.parquet.compression.codec","gzip")

In [4]:
# Read 2023-2024 TLC data
df_2023 = spark.read.parquet('../data/landing/tlc_data/2023/*.parquet')
df_2024 = spark.read.parquet('../data/landing/tlc_data/2024/*.parquet')

In [5]:
# Read NYC weather data 
wdata = pd.read_csv('../data/landing/external/NYC weather.csv')
wdata.head(10)

,,,,,,"NY CITY CENTRAL PARK, NY US (USW00094728)"
Date,TAVG (Degrees Fahrenheit),TMAX (Degrees Fahrenheit),TMIN (Degrees Fahrenheit),PRCP (Inches),SNOW (Inches),SNWD (Inches)
1869-01-01,NaN,29,19,0.75,9.0,NaN
1869-01-02,NaN,27,21,0.03,0.0,NaN
1869-01-03,NaN,35,27,0.00,0.0,NaN
1869-01-04,NaN,37,34,0.18,0.0,NaN
1869-01-05,NaN,43,37,0.05,0.0,NaN
1869-01-06,NaN,38,34,0.00,0.0,NaN
1869-01-07,NaN,48,35,0.00,0.0,NaN
1869-01-08,NaN,54,40,0.00,0.0,NaN
1869-01-09,NaN,48,38,0.00,0.0,NaN


In [6]:
wdata.tail(10)

,,,,,,"NY CITY CENTRAL PARK, NY US (USW00094728)"
2024-08-08,NaN,70,65,0.36,0.0,0.0
2024-08-09,NaN,80,70,0.11,0.0,0.0
2024-08-10,NaN,85,73,0.00,0.0,0.0
2024-08-11,NaN,80,69,0.00,0.0,0.0
2024-08-12,NaN,81,66,0.00,0.0,0.0
2024-08-13,NaN,81,64,0.00,0.0,0.0
2024-08-14,NaN,85,67,0.00,0.0,0.0
2024-08-15,NaN,86,68,0.00,0.0,0.0
2024-08-16,NaN,87,70,0.00,0.0,0.0
2024-08-17,NaN,79,67,0.04,0.0,0.0


In [7]:
# Read traffic data
tdata = pd.read_csv('../data/landing/external/NYC traffic.csv')
tdata.head(10)

,id,segmentid,roadway_name,from,to,direction,date,_12_00_1_00_am,_1_00_2_00am,_2_00_3_00am,...,_2_00_3_00pm,_3_00_4_00pm,_4_00_5_00pm,_5_00_6_00pm,_6_00_7_00pm,_7_00_8_00pm,_8_00_9_00pm,_9_00_10_00pm,_10_00_11_00pm,_11_00_12_00am
0,1,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,2012-01-09T00:00:00.000,20.0,10.0,11.0,...,104.0,105.0,147.0,120.0,91.0,83.0,74.0,49.0,42.0,42.0
1,2,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,2012-01-10T00:00:00.000,21.0,16.0,8.0,...,102.0,98.0,133.0,131.0,95.0,73.0,70.0,63.0,42.0,35.0
2,3,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,2012-01-11T00:00:00.000,27.0,14.0,6.0,...,115.0,115.0,130.0,143.0,106.0,89.0,68.0,64.0,56.0,43.0
3,4,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,2012-01-12T00:00:00.000,22.0,7.0,7.0,...,71.0,127.0,122.0,144.0,122.0,76.0,64.0,58.0,64.0,43.0
4,5,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,2012-01-13T00:00:00.000,31.0,17.0,7.0,...,113.0,126.0,133.0,135.0,102.0,106.0,58.0,58.0,55.0,54.0
5,6,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,2012-01-14T00:00:00.000,42.0,27.0,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,SB,2012-01-09T00:00:00.000,27.0,12.0,12.0,...,165.0,197.0,152.0,174.0,128.0,95.0,87.0,73.0,57.0,42.0
7,8,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,SB,2012-01-10T00:00:00.000,26.0,16.0,11.0,...,192.0,180.0,161.0,171.0,120.0,96.0,90.0,70.0,63.0,49.0
8,9,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,SB,2012-01-11T00:00:00.000,32.0,16.0,8.0,...,178.0,194.0,168.0,160.0,143.0,114.0,78.0,64.0,49.0,45.0
9,10,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,SB,2012-01-12T00:00:00.000,24.0,12.0,7.0,...,171.0,184.0,157.0,167.0,148.0,112.0,86.0,77.0,63.0,51.0


In [8]:
path = "../data/landing/tlc_data"
df_2023_5 = spark.read.parquet(path + "/2023/2023-05.parquet")
df_2024_1 = spark.read.parquet(path + "/2024/2024-01.parquet")

In [9]:
path = "../data/landing/tlc_data"
gdf_2023_5 = spark.read.parquet(path + "/2023/G-2023-05.parquet")
gdf_2024_1 = spark.read.parquet(path + "/2024/G-2024-01.parquet")

In [10]:
# Show 2023-5 yellow taxi data 
df_2023_5.show(10)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2023-05-01 00:33:13|  2023-05-01 00:53:01|              0|          7.8|         1|                 N|         138|          43|           1|       33.8| 7.75|    0.5|       8.

24/08/22 19:31:04 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 407567 ms exceeds timeout 120000 ms
24/08/22 19:31:04 WARN SparkContext: Killing executors is not supported by current scheduler.
24/08/22 19:31:04 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at o

In [21]:
# Show 2023-5 green taxi data
gdf_2023_5.show(10)

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2023-01-01 00:26:10|  2023-01-01 00:37:11|                 N|       1.0|         166|         143|            1.0|         2.58|       14.9|  1.0|    0.

24/08/22 04:23:22 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1081642 ms exceeds timeout 120000 ms
24/08/22 04:23:22 WARN SparkContext: Killing executors is not supported by current scheduler.
24/08/22 04:23:23 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$

## Preprocessing

In [19]:
# Filtering the data
df_2023_filtered = df_2019_1.filter(df_2019_1["trip_distance"] > 10) 
df_2023_filtered.show(5)

NameError: name 'df_2019_1' is not defined

In [ ]:
df_2019_filtered.where(F.col('passenger_count') > 3).limit(10)

In [2]:
# 合并2019年和2021年的数据
 df_all = df_2019_1.union(df_2021_1)
 df_all.show(10)

IndentationError: unexpected indent (3649347377.py, line 2)

## Add new column ##

In [3]:
df_renamed = df_all.withColumnRenamed("RatecodeID", "Rate_codeID") 
df_renamed.show(5)

NameError: name 'df_all' is not defined

In [4]:
df_dropped = df_all.drop("passenger_count_plus_10")
df_dropped.show(5) 

NameError: name 'df_all' is not defined

In [5]:
df_all.groupBy("VendorID").agg({"passenger_count": "avg", "extra": "max"}).show()

NameError: name 'df_all' is not defined

## Sampling Data

In [6]:
SAMPLE_SIZE = 0.01

In [7]:
df = df_all.sample(SAMPLE_SIZE, seed = 20020223).toPandas() 
df

NameError: name 'df_all' is not defined

In [9]:
df_2019_1.printSchema() 
print(f"数据总量: {df_2019_1.count()}") 
df_2019_1.describe().show()

NameError: name 'df_2019_1' is not defined

## Handling missing data 

In [10]:
df_2019_1 = df_2019_1.dropna()
df_2019_1 = df_2019_1.fillna({'passenger_count': 1, 'trip_distance': 0.0})

NameError: name 'df_2019_1' is not defined

 ## Datatype Conversion

In [11]:
df_2019_1 = df_2019_1.withColumn("passenger_count", col("passenger_count").cast("integer"))
df_2019_1 = df_2019_1.withColumn("tpep_pickup_datetime", col("tpep_pickup_datetime")) 
df_2019_1 = df_2019_1.withColumn("tpep_dropoff_datetime", col("tpep_dropoff_datetime"))  

NameError: name 'df_2019_1' is not defined

## Handling duplicates

In [12]:
df_2019_1 = df_2019_1.dropDuplicates()

NameError: name 'df_2019_1' is not defined

## Anomaly 

In [14]:
trip_distance_min = 0
trip_distance_max = 100
df_2019_1 = df_2019_1.filter((df_2019_1["trip_distance"] > trip_distance_min) & (df_2019_1["trip_distance"] > trip_distance_max))

NameError: name 'df_2019_1' is not defined

## Standardization 

In [15]:
from pyspark.ml.feature import StandardScaler, VectorAssembler
assembler = VectorAssembler(inputCols=["trip_distance", "trip_duration"], outputCol= "features")
df_features = assembler.transform(df_2019_1)
scaler = StandardScaler(inputCol="features", outputCol="scaled_features") 
scaler_model = scaler.fit(df_features)
df_scaled = scaler_model.transform(df_features)

AssertionError: 